# 第7章 型や型クラスを自分で作ろう

めっちゃ長い。だるい。資料作成終わってない。

## 7.1 新しいデータ型を定義する

## 7.2 形づくる

形を表す型を作ろう！

In [2]:
data Shape = Circle Float Float Float |
             Rectangle Float Float Float Float

In [3]:
:t Circle

Circle :: Float -> Float -> Float -> Shape

In [4]:
:t Rectangle

Rectangle :: Float -> Float -> Float -> Float -> Shape

面積計算したい。

In [5]:
area :: Shape -> Float
area (Circle _ _ r) = pi * r ^ 2
area (Rectangle x1 y1 x2 y2) = (abs $ x2 - x1) * (abs $ y2 - y1)

In [6]:
area $ Circle 10 20 10

314.15927

In [7]:
area $ Rectangle 0 0 100 100

10000.0

Shape型は表示できない。そんなときはShow型クラス。

In [8]:
data Shape = Circle Float Float Float |
             Rectangle Float Float Float Float
    deriving (Show) -- 勝手に実装してくれる。

In [9]:
Circle 10 20 5

Circle 10.0 20.0 5.0

In [10]:
Rectangle 50 230 60 90

Rectangle 50.0 230.0 60.0 90.0

In [11]:
map (Circle 10 20) [4,5,6,6]

[Circle 10.0 20.0 4.0,Circle 10.0 20.0 5.0,Circle 10.0 20.0 6.0,Circle 10.0 20.0 6.0]

### Pointデータ型で形を整える

`Float` の羅列じゃ座標なのか半径なのかよく分からん。

In [12]:
data Point = Point Float Float deriving (Show)
data Shape = Circle Point Float |
             Rectangle Point Point
    deriving (Show)

In [13]:
area :: Shape -> Float
area (Circle _ r) = pi * r ^ 2
area (Rectangle (Point x1 y1) (Point x2 y2)) = (abs $ x2 - x1) * (abs $ (y2 - y1))

In [14]:
area (Rectangle (Point 0 0) (Point 100 100))

10000.0

In [15]:
area (Circle (Point 0 0) 24)

1809.5574

In [16]:
nudge :: Shape -> Float -> Float -> Shape
nudge (Circle (Point x y) r) a b
    = Circle (Point (x+a) (y+b)) r
nudge (Rectangle (Point x1 y1) (Point x2 y2)) a b
    = Rectangle (Point (x1+a) (y1+b)) (Point (x2+a) (y2+b))

In [17]:
nudge (Circle (Point 34 34) 10) 5 10

Circle (Point 39.0 44.0) 10.0

やったね！

いちいち `Point x y` とかやるのがかったるい。

In [18]:
baseCircle :: Float -> Shape
baseCircle r = Circle (Point 0 0) r

baseRect :: Float -> Float -> Shape
baseRect width height = Rectangle (Point 0 0) (Point width height)

In [19]:
nudge (baseRect 40 100) 60 23

Rectangle (Point 60.0 23.0) (Point 100.0 123.0)

### Shapeをモジュールとしてエクスポートする

In [20]:
:load src/Chapter7/Shapes.hs
import Shapes

In [21]:
print $ Circle (Point 10 20) 30

Not in scope: data constructor `Circle'


Not in scope: data constructor `Point'

In [22]:
print $ nudge (baseCircle 30) 10 20

Circle (Point 10.0 20.0) 30.0

## 7.3 レコード構文

In [23]:
data Person = Person String String Int Float String String
    deriving (Show)

let guy = Person "Buddy" "Finklestein" 43 184.2 "526-2928" "Chocolate"
guy

Person "Buddy" "Finklestein" 43 184.2 "526-2928" "Chocolate"

In [24]:
firstName :: Person -> String
firstName (Person firstname _ _ _ _ _) = firstname

lastName :: Person -> String
lastName (Person _ lastname _ _ _ _) = lastname

age :: Person -> Int
age (Person _ _ age _ _ _) = age

height :: Person -> Float
height (Person _ _ _ height _ _) = height

phoneNumber :: Person -> String
phoneNumber (Person _ _ _ _ number _) = number

flavor :: Person -> String
flavor (Person _ _ _ _ _ flavor) = flavor

In [25]:
firstName guy

"Buddy"

In [26]:
height guy

184.2

In [27]:
flavor guy

"Chocolate"

こんなコードは嫌だ！

In [28]:
data Person = Person { firstNmae :: String
                     , lastName :: String
                     , age :: Int
                     , height :: Float
                     , phoneNumber :: String
                     , flavor :: String } deriving (Show)

In [29]:
:t flavor

flavor :: Person -> String

In [30]:
:t firstName

firstName :: Person -> String

In [31]:
data Car = Car String String Int deriving (Show)

In [32]:
Car "Ford" "Mustang" 1967

Car "Ford" "Mustang" 1967

In [33]:
data Car = Car { company :: String
               , model :: String
               , year :: Int
               } deriving (Show)

In [34]:
Car {company="Ford", model="Mustang", year=1967}

Car {company = "Ford", model = "Mustang", year = 1967}

In [155]:
Car "Ford" "Mustang" 1967

Car {company = "Ford", model = "Mustang", year = 1967}

## 7.4 型引数

In [35]:
Just "Haha"

Just "Haha"

In [36]:
Just 84

Just 84

In [37]:
:t Just "Haha"

Just "Haha" :: Maybe [Char]

In [38]:
:t Just 84

Just 84 :: forall a. Num a => Maybe a

In [39]:
:t Nothing

Nothing :: forall a. Maybe a

In [40]:
Just 10 :: Maybe Double

Just 10.0

In [41]:
data IntMaybe = INothing | IJust Int
data StringMaybe = SNothing | SJust String
data ShapeMaybe = ShNothing | ShJust Shape

とかするのは面倒くさい。

In [42]:
data Car a b c = Car { company :: a
                     , model :: b
                     , year :: c
                     } deriving (Show)

アホらしい。

In [43]:
tellCar :: (Show a) => Car String String a -> String
tellCar (Car {company = c, model = m, year = y})
    = "This "++ c ++ " " ++ m ++ " was made in " ++ show y

In [44]:
tellCar (Car "Ford" "Mustang" 1967)

"This Ford Mustang was made in 1967"

In [45]:
tellCar (Car "Ford" "Mustang" "nineteen sixty seven")

"This Ford Mustang was made in \"nineteen sixty seven\""

In [46]:
:t Car "Ford" "Mustang" 1967

Car "Ford" "Mustang" 1967 :: forall c. Num c => Car [Char] [Char] c

In [47]:
:t Car "Ford" "Mustang" "nineteen sixty seven"

Car "Ford" "Mustang" "nineteen sixty seven" :: Car [Char] [Char] [Char]

### 三次元ベクトル

In [161]:
data Vector a = Vector a a a deriving (Show)

vplus :: (Num a) => Vector a -> Vector a -> Vector a
(Vector i j k) `vplus` (Vector l m n) = Vector (i+l) (j+m) (k+n)

dotProd :: (Num a) => Vector a -> Vector a -> a
(Vector i j k) `dotProd` (Vector l m n) = i*l + j*m + k*n

vmult :: (Num a) => Vector a -> a -> Vector a
(Vector i j k) `vmult` m = Vector (i*m) (j*m) (k*m)

In [49]:
Vector 3 5 8 `vplus` Vector 9 2 8

Vector 12 7 16

In [50]:
Vector 3 5 8 `vplus` Vector 9 2 8 `vplus` Vector 0 2 3

Vector 12 9 19

In [51]:
Vector 3 9 7 `vmult` 10

Vector 30 90 70

In [52]:
Vector 4 9 5 `dotProd` Vector 9.0 2.0 4.0

74.0

In [53]:
Vector 2 9 3 `vmult` (Vector 4 9 5 `dotProd` Vector 9 2 4)

Vector 148 666 222

## 7.5 インスタンスの自動導出

### 人間の平等

In [54]:
data Person = Person { firstName :: String
                     , lastName :: String
                     , age :: Int
                     } deriving (Eq) -- 等値比較

In [55]:
let mikeD = Person { firstName = "Michael", lastName = "Diamond", age = 43 }
let adRock = Person { firstName = "Adam", lastName = "Horovitz", age = 41 }
let mca = Person { firstName = "Adam", lastName = "Yauch", age = 44 }

In [56]:
mca == adRock

False

In [57]:
mikeD == adRock

False

In [58]:
mikeD == mikeD

True

In [59]:
mikeD == Person { firstName = "Michael", lastName = "Diamond", age = 43 }

True

In [60]:
let beastieBoys = [mca, adRock, mikeD]
mikeD `elem` beastieBoys

True

### 読み方をかいてみせてよ

In [61]:
data Person = Person { firstName :: String
                     , lastName :: String
                     , age :: Int
                     } deriving (Eq, Show, Read)

In [62]:
let mikeD = Person { firstName = "Michael", lastName = "Diamond", age = 43 }
let adRock = Person { firstName = "Adam", lastName = "Horovitz", age = 41 }
let mca = Person { firstName = "Adam", lastName = "Yauch", age = 44 }

In [63]:
mikeD

Person {firstName = "Michael", lastName = "Diamond", age = 43}

In [64]:
"mikeD is: " ++ show mikeD

"mikeD is: Person {firstName = \"Michael\", lastName = \"Diamond\", age = 43}"

In [65]:
mysteryDude = "Person { firstName = \"Michael\"" ++
                     ", lastName = \"Diamond\"" ++
                     ", age = 43 }"
read mysteryDude :: Person

Person {firstName = "Michael", lastName = "Diamond", age = 43}

In [66]:
read mysteryDude == mikeD

True

In [67]:
read "Just 3" :: Maybe a

No instance for (Read a1) arising from a use of `read'
Possible fix: add (Read a1) to the context of an expression type signature: Maybe a1
In the expression: read "Just 3" :: Maybe a
In an equation for `it': it = read "Just 3" :: Maybe a

In [68]:
read "Just 3" :: Maybe Int

Just 3

### 順番を守ってください！

In [70]:
True `compare` False

GT

In [71]:
True > False

True

In [72]:
True < False

False

In [73]:
Nothing < Just 100

True

In [74]:
Nothing > Just (-49999)

False

In [75]:
Just 3 `compare` Just 2

GT

In [76]:
Just 100 > Just 50

True

### 何曜日でもいいよ

In [77]:
data Day = Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday
    deriving (Eq, Ord, Show, Read, Bounded, Enum)

In [78]:
Wednesday

Wednesday

In [79]:
show Wednesday

"Wednesday"

In [80]:
read "Saturday" :: Day

Saturday

In [81]:
Saturday == Sunday

False

In [82]:
Saturday == Saturday

True

In [83]:
Saturday > Friday

True

In [84]:
Monday `compare` Wednesday

LT

In [85]:
(minBound :: Day)

Line 1: Redundant bracket
Found:
(minBound :: Day)
Why not:
minBound :: Day

Monday

In [86]:
(maxBound :: Day)

Line 1: Redundant bracket
Found:
(maxBound :: Day)
Why not:
maxBound :: Day

Sunday

In [87]:
succ Monday

Tuesday

In [88]:
pred Saturday

Friday

In [89]:
[Thursday .. Sunday]

[Thursday,Friday,Saturday,Sunday]

In [90]:
[minBound .. maxBound] :: [Day]

[Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday]

## 7.6 型シノニム

In [91]:
type String = [Char]

Line 1: Use String
Found:
[Char]
Why not:
String

In [93]:
phoneBook :: [(String, String)]
phoneBook =
    [("betty", "555-2938")
    ,("bonnie", "452-2928")
    ,("patsy", "493-2928")
    ,("lucille", "205-2928")
    ,("wendy", "939-8282")
    ,("penny", "853-2492")]

In [94]:
type PhoneBook = [(String, String)]

In [95]:
type PhoneNumber = String
type Name = String
type PhoneBook = [(Name, PhoneNumber)]

In [96]:
inPhoneBook :: Name -> PhoneNumber -> PhoneBook -> Bool
inPhoneBook name pnumber pbook = (name, pnumber) `elem` pbook

### 型シノニムの多相化

In [97]:
type AssocList k v = [(k, v)]

In [98]:
import qualified Data.Map as Map
-- type IntMap v = Map.Map Int v
type IntMap = Map.Map Int

### そこを左に行って、すぐ右へ

In [100]:
Right 20

Right 20

In [101]:
Left "w00t"

Left "w00t"

In [102]:
:t Right 'a'

Right 'a' :: forall a. Either a Char

In [103]:
:t Left True

Left True :: forall b. Either Bool b

In [104]:
import qualified Data.Map as Map

data LockerState = Taken | Free deriving (Show, Eq)

type Code = String

type LockerMap = Map.Map Int (LockerState, Code)

In [105]:
lockerLookup :: Int -> LockerMap -> Either String Code
lockerLookup lockerNumber map = case Map.lookup lockerNumber map of
    Nothing -> Left $ "Locker " ++ show lockerNumber ++ " doesn't exist!"
    Just (state, code) -> if state /= Taken
                            then Right code
                            else Left $ "Locker " ++ show lockerNumber ++ " is already taken!"

In [106]:
lockers :: LockerMap
lockers = Map.fromList
    [(100, (Taken, "XD39I"))
    ,(101, (Free, "JAH3I"))
    ,(103, (Free, "IQSA9"))
    ,(105, (Free, "QOTSA"))
    ,(109, (Taken, "893JJ"))
    ,(110, (Taken, "99292"))]

In [107]:
lockerLookup 101 lockers

Right "JAH3I"

In [108]:
lockerLookup 100 lockers

Left "Locker 100 is already taken!"

In [109]:
lockerLookup 102 lockers

Left "Locker 102 doesn't exist!"

In [110]:
lockerLookup 110 lockers

Left "Locker 110 is already taken!"

In [111]:
lockerLookup 105 lockers

Right "QOTSA"

## 7.7 再帰的なデータ構造

In [112]:
data List a = Empty | Cons a (List a) deriving (Show, Read, Eq, Ord)

In [113]:
Empty

Empty

In [114]:
5 `Cons` Empty

Cons 5 Empty

In [115]:
4 `Cons` (5 `Cons` Empty)

Cons 4 (Cons 5 Empty)

In [116]:
3 `Cons` (4 `Cons` (5 `Cons` Empty))

Cons 3 (Cons 4 (Cons 5 Empty))

In [117]:
3 : (4 : (5 : []))

Line 1: Use list literal
Found:
3 : (4 : (5 : []))
Why not:
[3, 4, 5]

[3,4,5]

### リストの改善

In [118]:
infixr 5 :-:
data List a = Empty | a :-: (List a) deriving (Show, Read, Eq, Ord)

In [164]:
3 :-: Empty
-- 3 :-: (4 :-: (5 :-: Empty))

3 :-: Empty

In [120]:
-- let a = 3 :-: 4 :-: 5 :-: Empty
let a = 3 :-: (4 :-: (5 :-: Empty))
100 :-: a

Non type-variable argument in the constraint: Num (List a)
(Use FlexibleContexts to permit this)
When checking that `a' has the inferred type
  a :: forall a. (Num a, Num (List a), Num (List (List a))) => List (List (List a))

In [121]:
infixr 5 ^++
(^++) :: List a -> List a -> List a
Empty ^++ ys = ys
(x :-: xs) ^++ ys = x :-: (xs ^++ ys)

In [122]:
let a = 3 :-: (4 :-: (5 :-: Empty))
let b = 6 :-: (7 :-: Empty)
a ^++ b

3 :-: (4 :-: (5 :-: (6 :-: (7 :-: Empty))))

### 木を植えよう

In [123]:
data Tree a = EmptyTree | Node a (Tree a) (Tree a) deriving (Show)

## 7.8 型クラス 中級講座

### Eq型クラスの内部

In [124]:
class Eq a where
    (==) :: a -> a -> Bool
    (/=) :: a -> a -> Bool
    x == y = not (x /= y)
    x /= y = not (x == y)

Line 4: Use ==
Found:
not (x /= y)
Why not:
x == yLine 5: Use /=
Found:
not (x == y)
Why not:
x /= y

### 交通信号データ型

In [125]:
data TrafficLight = Red | Yellow | Green

In [126]:
instance Eq TrafficLight where
    Red == Red = True
    Green == Green = True
    Yellow == Yellow = True
    _ == _ = False

In [127]:
instance Show TrafficLight where
    show Red = "Red light"
    show Yellow = "Yellow light"
    show Green = "Green light"

In [128]:
Red == Red

True

In [129]:
Red == Yellow

False

In [165]:
Red `elem` [Red, Yellow, Green]

No instance for (IHaskellPrelude.Eq TrafficLight) arising from a use of `elem'
In the expression: elem Red [Red, Yellow, Green]
In an equation for `it': it = elem Red [Red, Yellow, Green]

In [131]:
[Red, Yellow, Green]

[Red light,Yellow light,Green light]

### サブクラス化

### 多相型を型クラスのインスタンスに

In [132]:
instance Eq (Maybe m) where
    Just x == Just y = x == y
    Nothing == Nothing = True
    _ == _ = False

No instance for (Eq m) arising from a use of `=='
Possible fix: add (Eq m) to the context of the instance declaration
In the expression: x == y
In an equation for `==': (Just x) == (Just y) = x == y
In the instance declaration for `Eq (Maybe m)'

In [133]:
instance (Eq m) => Eq (Maybe m) where
    Just x == Just y = x == y
    Nothing == Nothing = True
    _ == _ = False

## 7.9 YesとNoの型クラス

In [134]:
class YesNo a where
    yesno :: a -> Bool

In [135]:
instance YesNo Int where
    yesno 0 = False
    yesno _ = True

In [136]:
instance YesNo [a] where
    yesno [] = False
    yesno _ = True

In [137]:
instance YesNo Bool where
    yesno = id

In [138]:
instance YesNo (Maybe a) where
    yesno (Just _) = True
    yesno Nothing = False

In [139]:
instance YesNo (Tree a) where
    yesno EmptyTree = False
    yesno _ = True

In [140]:
instance YesNo TrafficLight where
    yesno Red = False
    yesno _ = True

In [141]:
yesno $ length []

Line 1: Evaluate
Found:
length []
Why not:
0

False

In [142]:
yesno "haha"

True

In [143]:
yesno ""

False

In [144]:
yesno $ Just 0

True

In [145]:
yesno True

True

In [146]:
yesno EmptyTree

False

In [147]:
yesno []

False

In [148]:
yesno [0,0,0]

True

In [149]:
:t yesno

yesno :: forall a. YesNo a => a -> Bool

## 7.10 Functor型クラス

In [150]:
fmap (*2) [1..3]

[2,4,6]

In [151]:
map (*2) [1..3]

[2,4,6]

In [168]:
fmap (+2) (Just 1)

Just 3

In [169]:
:info Functor

## 7.11 型を司るもの、種類

In [173]:
:kind Int

Int :: *

In [172]:
:type map

map :: forall a b. (a -> b) -> [a] -> [b]

In [174]:
:kind Functor

Functor :: (* -> *) -> Constraint

In [175]:
:kind Maybe

Maybe :: * -> *

In [176]:
:kind Either

Either :: * -> * -> *